In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3 MB 5.3 MB/s 


In [ ]:
import os
import subprocess
import tensorflow as tf
from mtcnn.mtcnn import MTCNN
import dlib
import cv2 as cv
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd

import torchvision.models.resnet as resnet
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/checkpoint.pth.tar' -d '/content/'

In [ ]:
!unzip '/content/drive/MyDrive/Archive.zip' -d '/content/DAiSEE Dataset/DAiSEE/Dataset/Test'
!unzip '/content/drive/MyDrive/Archive2.zip' -d '/content/DAiSEE Dataset/DAiSEE/Dataset/Test'
!unzip '/content/drive/MyDrive/Archive3.zip' -d '/content/DAiSEE Dataset/DAiSEE/Dataset/Valid'
!unzip '/content/drive/MyDrive/Archive4.zip' -d '/content/DAiSEE Dataset/DAiSEE/Dataset/Valid'
!unzip '/content/drive/MyDrive/998826.zip' -d '/content/DAiSEE Dataset/DAiSEE/Dataset/Valid'
# !mv "/content/DAiSEE Dataset/DAiSEE/Dataset/Test/987736" "/content/DAiSEE Dataset/DAiSEE/Dataset/Valid"
# !mv "/content/DAiSEE Dataset/DAiSEE/Dataset/Test/940328" "/content/DAiSEE Dataset/DAiSEE/Dataset/Valid"

In [ ]:
# Split a video and save each frame
def split_video(video_file, image_name_prefix, destination_path):
    return subprocess.check_output('ffmpeg -i "' + destination_path+video_file + '" ' + image_name_prefix + '%d.jpg -hide_banner', shell=True, cwd=destination_path)

In [ ]:
# Code for getting frame from each video

for ttv in ['Test', 'Train', 'Validation']:
    print('ttv is', ttv)
    if ttv == '.DS_Store':
        continue
    users = os.listdir('DAiSEE Dataset/DAiSEE/Dataset/'+ttv+'/')
    for user in users:
        print('user is', user)
        if user == '.DS_Store':
            continue
        currUser = os.listdir('DAiSEE Dataset/DAiSEE/Dataset/'+ttv+'/'+user+'/')
        for extract in currUser:
            print('extract is', extract)
            if extract == '.DS_Store':
                continue
            clip = os.listdir('DAiSEE Dataset/DAiSEE/Dataset/'+ttv+'/'+user+'/'+extract+'/')[0]
            if clip == '.DS_Store':
                clip = os.listdir('DAiSEE Dataset/DAiSEE/Dataset/'+ttv+'/'+user+'/'+extract+'/')[1]
            print (clip[:-4])
            path = os.path.abspath('.')+'/DAiSEE Dataset/DAiSEE/Dataset/'+ttv+'/'+user+'/'+extract+'/'
            split_video(clip, clip[:-4], path)
            # break

In [ ]:
# Move each frame into a separate directory

for x in ['Test', 'Valid']:
  for i in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x):
    if i not in ['__MACOSX','.ipynb_checkpoints']:
      for j in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'/'+i):
        if j == '.DS_Store':
          continue
        for k in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'/'+i+'/'+j):
          if k[-4:] == '.jpg':
            try:
              shutil.move('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'/'+i+'/'+j+'/'+k, '/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'Frames/'+i+'/'+j+'/')
            except IOError as io_err:
              os.makedirs(os.path.dirname('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'Frames/'+i+'/'+j+'/'))
              shutil.move('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'/'+i+'/'+j+'/'+k, '/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'Frames/'+i+'/'+j+'/')

In [ ]:
# !rm -rf '/content/DAiSEE Dataset/DAiSEE/Dataset/Train/'
# !rm -rf '/content/DAiSEE Dataset/DAiSEE/Dataset/Test/'
# !rm -rf '/content/DAiSEE Dataset/DAiSEE/Dataset/Valid/'

In [ ]:
# !rm -rf '/content/DAiSEE Dataset/DAiSEE/Dataset2/TestFaces/'

In [ ]:
# Extract a single face from a given photograph

def extract_face(filename, required_size=(160, 160)):
    image = Image.open(filename)
    image = image.convert('RGB')
    pixels = np.asarray(image)
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

In [ ]:
# Run the FaceNet model on each video on different parts of the 10-sec video

cnt = 0
for x in ['TestFrames','ValidFrames']:
  for i in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x):
    if i not in ['__MACOSX','.ipynb_checkpoints']:
      for j in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'/'+i):
        if j == '.DS_Store':
          continue
        print('At file', j)
        cnt = 0
        for k in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'/'+i+'/'+j):
          cnt = cnt+1
          if cnt in [15,135,285]:
            pixels = extract_face('/content/DAiSEE Dataset/DAiSEE/Dataset/'+x+'/'+i+'/'+j+'/'+k)
            im = Image.fromarray(pixels)
            try:
              im.save('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x[0:-6]+'Faces/'+i+'/'+j+'/'+k)
            except IOError as io_err:
              os.makedirs(os.path.dirname('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x[0:-6]+'Faces/'+i+'/'+j+'/'))
              im.save('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x[0:-6]+'Faces/'+i+'/'+j+'/'+k)

In [ ]:
!unzip '/content/drive/MyDrive/TrainFaces.zip'
!unzip '/content/drive/MyDrive/TestFaces.zip'
!unzip '/content/drive/MyDrive/ValidFaces.zip'

!mv "/content/content/DAiSEE Dataset/DAiSEE/Dataset/TestFaces" "/content/DAiSEE Dataset/DAiSEE/Dataset2"
!mv "/content/content/DAiSEE Dataset/DAiSEE/Dataset/TrainFaces" "/content/DAiSEE Dataset/DAiSEE/Dataset2"
!mv "/content/content/DAiSEE Dataset/DAiSEE/Dataset/ValidFaces" "/content/DAiSEE Dataset/DAiSEE/Dataset2"

In [ ]:
# This file is part of EmotionNet2 a system for predicting facial emotions
# Copyright (C) 2018  Maeve Kennedy
# 
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# 
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
# 
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>

import torchvision.models.resnet as resnet
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import shutil
# For windowing
import numpy as np
import matplotlib.pyplot as plt

# For one file pass
import tempfile
# import face_detector
import os.path
from PIL import Image
# import extract_faces

import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.DEBUG)


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class Confusion():
    def __init__(self, loader):
        self.classes = loader.dataset.classes
        self.class_to_idx = loader.dataset.class_to_idx
        self.count = np.zeros((len(self.classes), len(self.classes)))
        # Rows are inputs, cols are outputs

    def add(self, output, input):
        output = output.cpu().data.numpy()
        input = input.cpu().numpy()
        for i, o in enumerate(np.argmax(output, axis=1)):
            self.count[input[i], o] += 1

    def print_confusion(self):
        out = np.copy(self.count)
        col_sums = out.sum(axis=1)
        out = out / col_sums[:, np.newaxis]
        print(out)

    def print_latex(self, avg):
        out = np.copy(self.count)
        col_sums = out.sum(axis=1)
        out = out / col_sums[:, np.newaxis]
        # Print LaTeX style
        print('\\begin{tabular}{' + 'l'*(len(self.classes)+1) + '}')
        print('\\hline')
        print('\t & ', end='')
        print(*self.classes, sep=' & ', end='\\\\\n')
        print('\\hline')
        for ri, r in enumerate(out):
            print('\t{} & '.format(self.classes[ri]), end='')
            for ci, c in enumerate(r):
                print('{0:.3f} {1}'.format(c * 100, '& ' if ci != len(r)-1 else '\\\\'),
                     end=('' if ci != len(r)-1 else '\n'))
        print('\\hline')
        print('\\end{tabular} \\\\')
        print('Average: {:.3}\\%'.format(avg))
                

class EmotionNet():
    def __init__(self, layers=[3, 4, 6, 3]):
        block = resnet.BasicBlock
        num_classes = 4
        self.model = resnet.ResNet(block, layers, num_classes)
        if torch.cuda.is_available():
            self.model.cuda()
        self.bestaccur = 0.0

    def save_checkpoint(self, is_best, filename='checkpoint.pth.tar'):
        torch.save(self.model.state_dict(), filename)
        if is_best:
            shutil.copyfile(filename, 'best-' + filename)

    def load_checkpoint(self, filename):
        sample_model = resnet.ResNet(resnet.BasicBlock, [3, 4, 23, 3], 7)
        if torch.cuda.is_available():
            sample_model.load_state_dict(torch.load(filename))
        else:
            sample_model.load_state_dict(torch.load(filename, map_location=torch.device('cpu')))
        sample_model.fc = nn.Linear(512, 4)
        self.model.load_state_dict(sample_model.state_dict())
    def test_model_show(self, testdir, show=False):
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        testdata = datasets.ImageFolder(testdir, transforms.Compose([
            transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
        ]))
        criterion = nn.CrossEntropyLoss()
        if torch.cuda.is_available():
            criterion = criterion.cuda()
        # Test
        self.model.eval()
        losses = AverageMeter()
        top1 = AverageMeter()
        for i, (input, target) in enumerate(testdata):
            img = np.copy(input.numpy())
            target_out = torch.LongTensor(1, 1).zero_()
            target_out[0][0] = target
            if torch.cuda.is_available():
                target_out = target_out.cuda(non_blocking=True)
            input_var = Variable(normalize(input).view(1, *input.size()), volatile=True)
            if torch.cuda.is_available():
                input_var = input_var.cuda()

            output = self.model(input_var)
            prec1 = accuracy(output.data, target_out, topk=(1,))
            top1.update(prec1[0][0], input.size(0))
            img = np.rollaxis(img, 0, 3)
            plt.title(prec1[0].cpu().numpy()[0])
            plt.imshow(img)
            plt.pause(0.01)
            
        print('Test, Prec: {}'.format(
              top1.avg))

    def train_model(self, datadir, validdir, outprefix, epochs=10, csvout=None):
        batch_size = 2
        num_workers = 4
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        train_loader = torch.utils.data.DataLoader(
            datasets.ImageFolder(datadir, transforms.Compose([
                transforms.RandomSizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ])),
            batch_size=batch_size, shuffle=True, num_workers=num_workers,
            pin_memory=torch.cuda.is_available()
        )

        valid_loader = torch.utils.data.DataLoader(
            datasets.ImageFolder(validdir, transforms.Compose([
                transforms.Scale(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])),
            batch_size=batch_size, shuffle=False, num_workers=num_workers,
            pin_memory=torch.cuda.is_available()
        )

        criterion = nn.CrossEntropyLoss()
        if torch.cuda.is_available():
            criterion = criterion.cuda()

        optimizer = torch.optim.Adam(self.model.parameters())
        # CSV
        if csvout is not None:
            csvf = open(csvout, 'w')

        # Switch to training
        for e in range(epochs):
            self.model.train()
            losses = AverageMeter()
            top1 = AverageMeter()
            # Train
            for i, (input, target) in enumerate(train_loader):
                if torch.cuda.is_available():
                    target = target.cuda(non_blocking=True)
                    input_var = Variable(input).cuda()
                else:
                    input_var = Variable(input)
                target_var = Variable(target)
                output = self.model(input_var)

                loss = criterion(output, target_var)
                prec1 = accuracy(output.data, target, topk=(1,))
                losses.update(loss.data, input.size(0))
                top1.update(prec1[0][0], input.size(0))

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if i % 5 == 0:
                    logging.info('Epoch: {} {}/{}\nLoss: {}\nPrec: {}'.format(
                      e, i, len(train_loader), losses.avg, top1.avg))
            # Valid
            lossavg, topavg = self._valid_model(valid_loader)
                
            logging.info('Epoch: {}, Validation Loss: {}, Prec: {}'.format(
                  e, lossavg, topavg))
            if csvout is not None:
                # Epoch, training loss, training accuracy, valid lossavg, topavg
                print('{}, {}, {}, {}, {}'.format(e, losses.avg, top1.avg, 
                      lossavg, topavg), flush=True, file=csvf)
            top = False
            if self.bestaccur < top1.avg:
                self.bestaccur = top1.avg
                top = True
            self.save_checkpoint(top, filename=outprefix + '.pth.tar')
        if csvout is not None:
            csvf.close()
    
    def valid_model(self, validdir):
        batch_size = 16
        num_workers = 4
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

        valid_loader = torch.utils.data.DataLoader(
            datasets.ImageFolder(validdir, transforms.Compose([
                transforms.Scale(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])),
            batch_size=batch_size, shuffle=False, num_workers=num_workers,
            pin_memory=torch.cuda.is_available()
        )

        loss, acc = self._valid_model(valid_loader)
        logging.info('Loss: {}, Precision: {}'.format(loss, acc))
    
    def classify_one_image(self, imgf,
            classes=['afraid', 'angry', 'disgusted', 'happy', 'neutral', 'sad', 'surprised']):
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        transf = transforms.Compose([
                transforms.Scale(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
        ])
        # Face detection
        args = {}
        args['threshold'] = 0.0
        args['window'] = False
        args['ignore_multi'] = True
        args['grow'] = 10 
        args['resize'] = True
        args['row_resize'] = 512
        args['col_resize'] = 512
        args['min_proportion'] = 0.1
        
        with tempfile.TemporaryDirectory() as tempdir:
            args['o'] = tempdir
            # face_detector.transform(extract_faces.AttributeDict(args), [imgf])
            # cropped = Image.open(tempdir + '/' + os.path.basename(imgf))
        cropped = Image.open(imgf)
        cropped = transf(cropped)
        
        input_var = Variable(cropped.view(1, *cropped.shape))

        if torch.cuda.is_available():
            input_var = input_var.cuda()

        output = self.model.forward(input_var).cpu().data.numpy()
        softmax = np.exp(output) / np.sum(np.exp(output))
        clss = np.argmax(softmax)
        # print('clss is', clss)
        # print('softmax is', softmax)
        # print('max of softmax is', softmax.max())
        return clss, softmax.max()
        # fig = plt.figure()
        # plt.imshow(Image.open(imgf))
        # fig.subplots_adjust(bottom=0.2)
        # plt.figtext(0.1, 0.05, ', '.join(classes))
        # plt.figtext(0.1, 0.10, ', '.join(['{:.3}'.format(a) for a in softmax.reshape(-1)]))
        # plt.title(classes[clss])
        # plt.show()

    def _valid_model(self, valid_loader): 
        self.model.eval()
        losses = AverageMeter()
        top1 = AverageMeter()
        confusion = Confusion(valid_loader)
        criterion = nn.CrossEntropyLoss()

        if torch.cuda.is_available():
            criterion = criterion.cuda()

        for i, (input, target) in enumerate(valid_loader):
            if torch.cuda.is_available():
                target = target.cuda(non_blocking=True)
            input_var = Variable(input, volatile=True)
            if torch.cuda.is_available():
                input_var = input_var.cuda()
            target_var = Variable(target, volatile=True)

            output = self.model(input_var)
            confusion.add(output, target)
            loss = criterion(output, target_var)

            prec1 = accuracy(output.data, target, topk=(1,))
            losses.update(loss.data, input.size(0))
            top1.update(prec1[0][0], input.size(0))
        print('Confusion Matrix')
        confusion.print_confusion()
        confusion.print_latex(top1.avg)
        return losses.avg, top1.avg

In [ ]:
net = EmotionNet(layers=[3, 4, 23, 3])
net.load_checkpoint('/content/checkpoint.pth.tar')

net.classify_one_image('/content/DAiSEE Dataset/DAiSEE/Dataset2/TrainFaces/510035/5100351001/5100351001112.jpg', classes=['boredom-0', 'boredom-1', 'boredom-2', 'boredom-3'])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


(2, 0.298013)

In [ ]:
results_frame = pd.DataFrame(columns= ['Video ID', 'Frame ID', 'Boredom class', 'Boredom prob', 'Engagement class', 'Engagement prob', 'Confusion class', 'Confusion prob', 'Frustration class', 'Frustration prob'])

net = EmotionNet(layers=[3, 4, 23, 3])
net.load_checkpoint('checkpoint.pth.tar')

for x in ['TrainFaces', 'TestFaces', 'ValidFaces']:
  for i in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x):
    if i not in ['__MACOSX','.ipynb_checkpoints']:
      for j in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x+'/'+i):
        if j == '.DS_Store':
          continue
        print('At file', j)
        for k in os.listdir('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x+'/'+i+'/'+j):
          b_class, b_prob = net.classify_one_image('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x+'/'+i+'/'+j+'/'+k, classes=['boredom-0', 'boredom-1', 'boredom-2', 'boredom-3'])
          e_class, e_prob = net.classify_one_image('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x+'/'+i+'/'+j+'/'+k, classes=['engagement-0', 'engagement-1', 'engagement-2', 'engagement-3'])
          c_class, c_prob = net.classify_one_image('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x+'/'+i+'/'+j+'/'+k, classes=['confusion-0', 'confusion-1', 'confusion-2', 'confusion-3'])
          f_class, f_prob = net.classify_one_image('/content/DAiSEE Dataset/DAiSEE/Dataset2/'+x+'/'+i+'/'+j+'/'+k, classes=['frustration-0', 'frustration-1', 'frustration-2', 'frustration-3'])
          results_frame = results_frame.append({
              'Video ID': j,
              'Frame ID': k[0:-4],
              'Boredom class': b_class,
              'Boredom prob': b_prob,
              'Engagement class': e_class,
              'Engagement prob': e_prob,
              'Confusion class': c_class,
              'Confusion prob': c_prob,
              'Frustration class': f_class,
              'Frustration prob': f_prob},
              ignore_index = True)
# classify_one_image('/content/DAiSEE Dataset/DAiSEE/Dataset/Test/510035/5100351001/51003510011.jpg')

In [ ]:
# results_frame.to_csv('/content/results_frame.csv')

In [ ]:
results_frame = pd.read_csv('/content/results_frame.csv')

In [ ]:
results_frame['Video ID'].nunique()

239

In [ ]:
results = results_frame.groupby(['Video ID']).agg({
    'Boredom class': lambda x: x.mode().iloc[0],
    'Engagement class': lambda x: x.mode().iloc[0],
    'Confusion class': lambda x: x.mode().iloc[0],
    'Frustration class': lambda x: x.mode().iloc[0]
}).reset_index()

In [ ]:
results_frame.head()

,Unnamed: 0,Video ID,Frame ID,Boredom class,Boredom prob,Engagement class,Engagement prob,Confusion class,Confusion prob,Frustration class,Frustration prob
0,0,5100351010,5100351010133,2,0.288267,2,0.288267,2,0.288267,2,0.288267
1,1,5100351010,5100351010258,2,0.280239,2,0.280239,2,0.280239,2,0.280239
2,2,5100351010,5100351010150,2,0.275746,2,0.275746,2,0.275746,2,0.275746
3,3,5100351010,5100351010221,2,0.293650,2,0.293650,2,0.293650,2,0.293650
4,4,5100351010,510035101032,2,0.277615,2,0.277615,2,0.277615,2,0.277615


In [ ]:
actual_labels = pd.read_csv('/content/drive/MyDrive/TestLabels.csv')

In [ ]:
results.dtypes, actual_labels.dtypes

(Video ID             int64
 Boredom class        int64
 Engagement class     int64
 Confusion class      int64
 Frustration class    int64
 dtype: object, ClipID          object
 Boredom          int64
 Engagement       int64
 Confusion        int64
 Frustration      int64
 dtype: object)

In [ ]:
results['Video ID'] = results['Video ID'].astype(str)
actual_labels['ClipID'] = actual_labels['ClipID'].str[0:-4]
results_comparison = pd.merge(results, actual_labels, left_on = 'Video ID', right_on = 'ClipID', how = 'left')

In [ ]:
def top1accu(df, col1, col2):
  count = 0
  for index, row in df.iterrows():
    if row[col1] == row[col2]:
      count = count + 1
  return count*100/df.shape[0]

In [ ]:
print('Boredom Accuracy is ',top1accu(results_comparison, 'Boredom class', 'Boredom'))
print('Engagement Accuracy is ',top1accu(results_comparison, 'Engagement class', 'Engagement'))
print('Confusion Accuracy is ',top1accu(results_comparison, 'Confusion class', 'Confusion'))
print('Frustration Accuracy is ',top1accu(results_comparison, 'Frustration class', 'Frustration '))

Boredom Accuracy is  18.828451882845187
Engagement Accuracy is  51.88284518828452
Confusion Accuracy is  5.439330543933054
Frustration Accuracy is  2.092050209205021
